# RAG Workshop – Dutch Pension Law  
Author: *Generated on 2025-04-30*  

This notebook accompanies an internal workshop on **Retrieval‑Augmented Generation (RAG)** with Dutch pension‑related law texts from [wetten.overheid.nl](https://wetten.overheid.nl).  
You will need:

* Python 3.10+  
* `openai`, `azure‑identity`, `langchain`, `langchain‑community`, `langchain‑huggingface`,`langchain_openai`, `tiktoken`, `rdflib`, `faiss-cpu`,`numpy`,`scikit-learn`,`time`,`textwrap`

From an empty source folder in which you will run the notebook, run the following commands:

1. **Clone & install**
   ```bash
   git init
   python -m venv .venv && source .venv/bin/activate
   git clone https://github.com/iAMLAB-test/workshop_llm
   pip install -r requirements.txt
   ```

Using `config_template.yaml` fill it's contexts (provided seperately) and save under `config.yaml`:

2. **Configure environment**
   ```bash
   API_KEY_DECODER: [Enter your API key here]
   AZURE_ENDPOINT: [Enter your Azure endpoint here]
   API_VERSION_DECODER: [Enter your API version here]
   ```

In [ ]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


No pyvenv.cfg file


### Simple LLM call.

In [2]:
#Prompting API example.
from utils import gpt_4o_mini

query = "Say 'Hello world' in five different languages."
print(gpt_4o_mini(query).choices[0].message.content)

ModuleNotFoundError: No module named 'yaml'

## 1 – Layout of Dutch law texts  
We will inspect local `.txt` dumps in `data/docs/`. Each file contains **one complete law** (some > 60 k tokens).

Below we load the all three files and print a *single article* per file to understand the structure.

### Random article from input law texts.

Here we show, using the three law texts, one random article from each.

In [ ]:
from pathlib import Path
import random
from utils import split_articles, wrap_at_spaces

# Load a law text file.
DOC_PATH = Path("data/docs")
for law_file in sorted(DOC_PATH.glob("*.txt")):
    law_text = law_file.read_text(encoding="utf-8")
    articles = split_articles(law_text)  # drop preamble

    example_idx = random.randrange(len(articles))
    article_key = list(articles.keys())[example_idx]
    print(f"\n--- {law_file.name}, total articles {len(articles)} ---")
    print(article_key)
    print(wrap_at_spaces(articles[article_key],width=100))


--- Pensioenswet.txt, total articles 323 ---
Artikel 150i.
Implementatieplan  1.      In het implementatieplan legt de pensioenuitvoerder schriftelijk vast op
welke wijze voorbereidingen worden getroffen voor de uitvoering van de gewijzigde
pensioenovereenkomst en invulling zal worden gegeven aan de uitvoering van de gewijzigde
pensioenovereenkomst, alsmede de wijze waarop zal worden omgegaan met opgebouwde pensioenaanspraken
en pensioenrechten.  2.      Het implementatieplan bevat in ieder geval de volgende onderdelen:
a.      de technische uitvoerbaarheid van de pensioenovereenkomst;         b.      de kosten die
verband houden met de uitvoering van de pensioenovereenkomst;         c.      de risico’s die
verband houden met de uitvoering van de pensioenovereenkomst;         d.      de
risicobeheersmaatregelen die worden getroffen in verband met de uitvoering van de
pensioenovereenkomst;         e.      de wijze waarop zal worden omgegaan met opgebouwde
pensioenaanspraken en pensioen

### Count tokens per file.

Given our working dataset, we show the number of tokens per law text below.

In [ ]:
# Token count
from utils import count_tokens_in_docs

print(count_tokens_in_docs())

Pensioenswet.txt: 126338 tokens
Uitvoeringsbesluit loonbelasting 1965.txt: 20766 tokens
Wet op loonbelasting 1964.txt: 64790 tokens

Total tokens across all files: 211894


## 2 – Direct LLM retrieval vs. Article‑wise retrieval  
We compare:

1. **Whole‑law prompt** – push the complete text (~60 k tokens)                                                 → costly ❌ fast ✔️ accurate ❌  
2. **Chunked/article prompts** – iterate per article                                                            → costly ❌ slow ❌ accurate ✔️  
3. **RAG** - use article text embeddings to identify five most relevant articles before searching per article   → cheap ✔️ fast ✔️ accurate ✔️  
4. **Graph RAG** - Using a law text's linked-data structure                                                     → cheap ✔️ fast ✔️ accurate ✔️

We demonstrates examples 3 and 4 at the end of the notebook.

### 2.1. First, using 'naive' full text chunking in one large system prompt.

In [ ]:
import time
import json
from utils import gpt_4o_mini, llm_metrics,wrap_at_spaces

QUESTION = "Wat is de franchise, welk artikel gebruik je ervoor en wat zijn uitzonderingen op de wettelijke waarde? Hou het antwoord onder de 50 woorden."

# --- Whole law ---
with open("data/docs/Wet op loonbelasting 1964.txt", "r", encoding="utf-8") as f:
    law_text = f.read()
    start = time.time()
    response = gpt_4o_mini(user_message=QUESTION,law_text = law_text)

    elapsed = time.time()-start
    out_tokens = response.usage.completion_tokens
    in_tokens = response.usage.prompt_tokens
    print(f"Q {QUESTION}")
    print(f"A: {wrap_at_spaces(response.choices[0].message.content,100)}")
    print(f"\n--- Performance ---")
    print(json.dumps(llm_metrics("Hele wetstekst ingeladen", in_tokens, out_tokens, elapsed),indent=2))

Q Wat is de franchise, welk artikel gebruik je ervoor en wat zijn uitzonderingen op de wettelijke waarde? Hou het antwoord onder de 50 woorden.
A: De franchise bedraagt € 18.475 volgens artikel 18a, tweede lid. Uitzonderingen op de wettelijke
waarde gelden voor pensioenregelingen die niet voldoen aan de vereisten van de pensioenwet, of bij
tijdelijke inactiviteit van de werknemer.

--- Performance ---
{
  "scenario": "Hele wetstekst ingeladen",
  "input tokens": 48515,
  "output tokens": 53,
  "USD cost": 0.014618,
  "elapsed seconds": 3.584
}


#### Answer incorrect!

### 2.2. Second, by looping per article section.

In [ ]:
import time
import json
from utils import gpt_4o_mini, llm_metrics,wrap_at_spaces,split_articles

QUESTION = "Wat is de franchise, en wat zijn uitzonderingen op de wettelijke waarde? "+\
"Indien deze tekst hier geen expliciete informatie over geeft, antwoord met enkel 'None'. Hou het antwoord onder de 50 woorden."


start = time.time()
in_tokens,out_tokens = 0,0
answers = []
with open("data/docs/Wet op loonbelasting 1964.txt", "r", encoding="utf-8") as f:
    law_text = f.read()
    article_dict = split_articles(law_text)
    for key, law_article_text in article_dict.items():
        response = gpt_4o_mini(user_message=QUESTION,law_text = law_article_text)
        in_tokens += response.usage.prompt_tokens
        out_tokens += response.usage.completion_tokens
        if "None" in str(response.choices[0].message.content):
            continue
        answers.append((key,response.choices[0].message.content))
    elapsed = time.time()-start
    llm_metrics("Zoeken per artikel", in_tokens, out_tokens, elapsed)
    for key,ans in answers:
        print(f"Q {QUESTION}")
        print(f"A: {wrap_at_spaces(ans,100)}")
        print(f"\n--- Performance ---")
        print(json.dumps(llm_metrics("Hele wetstekst ingeladen", in_tokens, out_tokens, elapsed),indent=2))


Q Wat is de franchise, en wat zijn uitzonderingen op de wettelijke waarde? Indien deze tekst hier geen expliciete informatie over geeft, antwoord met enkel 'None'. Hou het antwoord onder de 50 woorden.
A: De franchise bedraagt € 18.475, maar kan bij ministeriële regeling worden vervangen door een ander
bedrag. Een lager bedrag kan worden vastgesteld indien een lager percentage per dienstjaar wordt
toegepast dan het percentage genoemd in het eerste lid.

--- Performance ---
{
  "scenario": "Hele wetstekst ingeladen",
  "input tokens": 59330,
  "output tokens": 434,
  "USD cost": 0.01832,
  "elapsed seconds": 48.723
}


#### Answer is correct, but takes a long time to find (50+ seconds).


**Main observations**

- The franchise value can be lower than € 18.475 given the premium percentage, this is only observed when looping over articles.
- Looping over articles is accurate but takes a long time (50+ seconds in contrast to approximately 3 seconds). Both methods are costly, token-wise.

## 3. Introduction to RAG


A word/sentence embedding is a numerical representation (using a vector) of it's semantic meaning. 

By creating vector-stores of texts first, one no longer has to rely on word-matching or brute force loop-searching to find the right article containing specific text
by simply searching over a small subset of articles with an embedding closest to the question at hand, which is RAG in a nutshell. 

Cosine similarity measures the (L2) distance between two vectors, in this case word/sentence embeddings. 
Values are between 0 and 1, with values closer to 1 indicating words/sentences that are similar.

### 3.1. Comparing embeddings ofwords/sentences.

Change the model parameters to observe differences across similarity metrics.

In [ ]:
import numpy as np
from utils import text_embedding_3_large
from sklearn.metrics.pairwise import cosine_similarity
from langchain.embeddings import HuggingFaceEmbeddings

# Adjustable parameters
model = "mxbai-embed-large-v1" #"mxbai-embed-large-v1" or "text_embedding_3_large"

assert model in ["mxbai-embed-large-v1","text_embedding_3_large"], "model must be either 'hf' or 'azure'"
mxbai_emb = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")
#Test one, same sentence, one different word.
print( "--- model:",model,"---")
text_tuple_list = list([("car","cat")]+[(f"cat {sentence}",f"kitten {sentence}") for sentence in ["","is a large animal", "is a large animal with much fur"]])
for t1,t2 in text_tuple_list:
    if model == "mxbai-embed-large-v1":
        encoder_response = mxbai_emb.embed_documents([t1,t2])
        v1,v2 = (np.array(encoder_response[i]).reshape(1,-1) for i in [0,1])
    else:
        encoder_response = text_embedding_3_large([t1,t2])
        v1,v2 = (np.array(encoder_response.data[i].embedding).reshape(1,-1) for i in [0,1])
    print(f"t1: {t1}")
    print(f"t2: {t2}")
    print("distance",np.round(cosine_similarity(v1,v2)[0,0],2))

#Test two, same meaning, different words.
print( "---")
text_tuple_list = list([("rock","object that beats scissors in rock paper scissors"),("Tallest building in New York in 1931","The Empire State Building")])
for t1,t2 in text_tuple_list:
    if model == "mxbai-embed-large-v1":
        encoder_response = mxbai_emb.embed_documents([t1,t2])
        v1,v2 = (np.array(encoder_response[i]).reshape(1,-1) for i in [0,1])
    else:
        encoder_response = text_embedding_3_large([t1,t2])
        v1,v2 = (np.array(encoder_response.data[i].embedding).reshape(1,-1) for i in [0,1])
    print(f"t1: {t1}")
    print(f"t2: {t2}")
    print("distance",np.round(cosine_similarity(v1,v2)[0,0],2))

--- model: mxbai-embed-large-v1 ---
t1: car
t2: cat
distance 0.7
t1: cat 
t2: kitten 
distance 0.83
t1: cat is a large animal
t2: kitten is a large animal
distance 0.89
t1: cat is a large animal with much fur
t2: kitten is a large animal with much fur
distance 0.91
---
t1: rock
t2: object that beats scissors in rock paper scissors
distance 0.56
t1: Tallest building in New York in 1931
t2: The Empire State Building
distance 0.68


**This cell demonstrates two things:**

When keeping one word different but increasing the sentence size, the differing word leads to a smaller difference between the resulting embeddings.

Distance between 'similar' sentences using the 'text_embedding_3_large' model seem to be further apart compared to the HuggingFaceEmbeddings 'mxbai-embed-large-v1' model.

This can be tested by adjusting the 'model' parameter above.

### 3.2. Creating law article vector stores using both models (takes 3+ minutes on my developer-laptop), do not rerun.

In [ ]:
# from pathlib import Path
# from langchain_community.vectorstores import FAISS
# import importlib,utils
# importlib.reload(utils)
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from utils import AZURE_EMBEDDINGS, MXBAI_EMBEDDINGS, split_articles
# import os

# dir_name = os.getcwd()

# # Save law texts seperately to a list and create the underlying vector_stores
# DOC_PATH = Path("data/docs")
# article_list = []
# for law_file in sorted(DOC_PATH.glob("*.txt")):
#     law_text = law_file.read_text(encoding="utf-8")
#     articles = split_articles(law_text)  # drop preamble
#     article_list += [f"""{law_file.name} {key} {value}""" for key,value in articles.items()]

# # 20000 chosen because the largest law text is 18000 characters.
# splitter = RecursiveCharacterTextSplitter(chunk_size=20000, chunk_overlap=0)
# documents = splitter.create_documents(article_list)

# # 'Vector stores', for now not stored efficiently (using FAISS langchain).
# mxbai_store = FAISS.from_documents(documents, MXBAI_EMBEDDINGS,normalize_L2=True)
# mxbai_store.save_local(os.path.join(dir_name,"data/vector_stores/mxbai-embed-large-v1_nongraph.index"))
# azure_store = FAISS.from_documents(documents, AZURE_EMBEDDINGS,normalize_L2=True)
# azure_store.save_local(os.path.join(dir_name,"data/vector_stores/text-embedding-3-large_nongraph.index"))


## 4. Demonstrating RAG using article vector store.

### 4.1. For the generated vector store shows 5 of the most closely matched articles to the original question.

In [ ]:
import textwrap
import importlib,utils
importlib.reload(utils)
from utils import MXBAI_STORE_NONGRAPH, AZURE_STORE_NONGRAPH, topk

# number of articles to check
n = 5

QUESTION = "Wat is de franchise, welk artikel gebruik je ervoor en wat zijn uitzonderingen op de wettelijke waarde? Hou het antwoord onder de 50 woorden."
v1 = topk(MXBAI_STORE_NONGRAPH,QUESTION,k=n)
v2 = topk(AZURE_STORE_NONGRAPH,QUESTION,k=n)

article_list_mxbai = [tup[0].page_content for tup in v1]
article_list_azure = [tup[0].page_content for tup in v2]

# Pair them distance-first so heapq.nlargest uses the distance as the key
dist_list = list(zip(["mixedbread-ai/mxbai-embed-large-v1","text_embedding_3_large"],[article_list_mxbai, article_list_azure]))
for name,article_list in dist_list:
    print(f"\n--- Top {n} --- {name}")
    for article in article_list:
        print(f"{textwrap.shorten(article,150)}")


--- Top 5 --- mixedbread-ai/mxbai-embed-large-v1
Wet op loonbelasting 1964.txt Artikel 38s 1. Indien een pensioenregeling op grond van artikel 150f van de Pensioenwet of artikel 145e van de Wet [...]
Wet op loonbelasting 1964.txt Artikel 18a 1. De premie per dienstjaar voor een ouderdomspensioen en een partnerpensioen bij overlijden op of na [...]
Pensioenswet.txt Artikel 130. Vermelding premie in jaarrekening en bestuursverslag [Vervallen per 01-07-2023]
Wet op loonbelasting 1964.txt Artikel 38m [Vervallen per 01-04-2017]
Wet op loonbelasting 1964.txt Artikel 38j [Vervallen per 01-04-2017]

--- Top 5 --- text_embedding_3_large
Wet op loonbelasting 1964.txt Artikel 18a 1. De premie per dienstjaar voor een ouderdomspensioen en een partnerpensioen bij overlijden op of na [...]
Pensioenswet.txt Artikel 150l. Standaard invaarpad 1. De wijze waarop wordt omgegaan met opgebouwde pensioenaanspraken en pensioenrechten als [...]
Wet op loonbelasting 1964.txt Artikel 13 1. Niet in geld genoten 

#### Article 18a of 'Wet op loonbelasting 1964' is the correct article.

The 'text_embedding_3_large' model ranks it first, 'mixedbread-ai/mxbai-embed-large-v1' model ranks it second.

### 4.2. Putting everything together, using mxbai encoder

In [ ]:
import time
import textwrap
import json
from langchain_community.callbacks import get_openai_callback
from utils import rag_executor, MXBAI_STORE_NONGRAPH

start = time.time()
QUESTION = "Wat is de franchise, welk artikel gebruik je ervoor en wat zijn uitzonderingen op de wettelijke waarde? Hou het antwoord onder de 50 woorden."
answer,top_docs,qa_chain = rag_executor(QUESTION,store = MXBAI_STORE_NONGRAPH)
print("Q:", QUESTION)
print("A:", answer)
print(f"\n--- Top {len(top_docs)} documents ---")
for doc in top_docs:
    print(f"{textwrap.shorten(str(doc.page_content),150)}")

print(f"\n--- Performance ---")
with get_openai_callback() as cb:
    answer = qa_chain.invoke({"input_documents": top_docs, "question": QUESTION})
    in_tokens = cb.prompt_tokens
    out_tokens = cb.completion_tokens
    elapsed = time.time()-start
print(json.dumps(llm_metrics("RAG op wetsartikelen", in_tokens, out_tokens, elapsed),indent=2))

Q: Wat is de franchise, welk artikel gebruik je ervoor en wat zijn uitzonderingen op de wettelijke waarde? Hou het antwoord onder de 50 woorden.
A: De franchise bedraagt € 18.475, zoals vermeld in Artikel 18a, lid 3 van de Wet op loonbelasting 1964. Uitzonderingen zijn mogelijk bij deeltijdwerk en indien een lager percentage per dienstjaar wordt toegepast.

--- Top 5 documents ---
Wet op loonbelasting 1964.txt Artikel 38s 1. Indien een pensioenregeling op grond van artikel 150f van de Pensioenwet of artikel 145e van de Wet [...]
Wet op loonbelasting 1964.txt Artikel 18a 1. De premie per dienstjaar voor een ouderdomspensioen en een partnerpensioen bij overlijden op of na [...]
Pensioenswet.txt Artikel 130. Vermelding premie in jaarrekening en bestuursverslag [Vervallen per 01-07-2023]
Wet op loonbelasting 1964.txt Artikel 38m [Vervallen per 01-04-2017]
Wet op loonbelasting 1964.txt Artikel 38j [Vervallen per 01-04-2017]

--- Performance ---
{
  "scenario": "RAG op wetsartikelen",
  "inp

#### Much faster and accurate compared to the methods in Section 2.1 or 2.2.

## 5. Graph RAG, taking advantage of linked data structure for RAG.

### 5.1. Law graph (created beforehand).

In [ ]:
#TODO: Placeholder, visualize graph.

### 5.2. Graph-based vector store.

In [ ]:
#TODO: Placeholder, create vector stores.

In [ ]:
#TODO: Placeholder, display top 5 articles based on a query.

### 5.3. Questions across articles.

In [ ]:
#TODO: RAG demonstration.

## Wrap‑up / Key takeaways ✅  

* **Direct prompting** on entire laws is cost‑heavy and hits context limits.  
* **Chunking** improves alignment but sacrifices latency.  
* **RAG** with a high‑quality embedding model (MXBAI) gives the *best accuracy‑per‑dollar*.  
* Integrating domain‑specific knowledge graphs can further boost recall for cross‑article references.  

Feel free to extend the notebook by  
* replacing placeholder accuracies with manual grading,  
* adding caching for embeddings,  
* deploying the FAISS index as an API.  
